Análisis del rendimiento de toda la comunidad que hacen dropshipping con SeventyBlock:
Para este análisis se desea determinar lo siguiente:

1. todas las comisiones logísticas sean pagadas, e imprimir en un excel todas las ordenes que no hayan sido pagadas por comision
2. todas las comisiones por referidos sean pagadas, e imprimir en un excel todas las ordenes que no hayan sido pagadas por comision



In [ ]:
ruta1 = './admin orden por fila oct 2023.xlsx'
ruta2 = './admin historial cartera Oct 2023.xlsx'
ruta3 = './proveedor historial de cartera oct 2023.xlsx'